In [1]:
import numpy as np
import os
import matlab.engine
import matplotlib.pyplot as plt
from Camera_util import parse_video, parse_mean_video
import pandas as pd
from natsort import natsorted
import time
import Camera_util 
import importlib
import pickle
import scipy.io
import heapq
from collections.abc import Iterable
from recorder import parse_settings_file

In [ ]:
path = '/home/eds/combiExperiments/code/lightLogger/raspberry_pi_firmware/test_settingsHistory.csv'

parse_settings_file(path)